In [1]:
import mysql.connector
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import os
from dotenv import load_dotenv
import unicodedata
import re
import pickle

In [2]:
pd.set_option("display.max_columns", None)

Load data from MySQL

In [3]:
load_dotenv()

# Event data
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    database=os.getenv("DB_NAME"),
    password=os.getenv("DB_PASSWORD")
    )

cursor = conn.cursor()

query = ("SELECT * FROM events")

cursor.execute(query)

columns = [desc[0] for desc in cursor.description]

rows = cursor.fetchall()

df_events = pd.DataFrame(rows, columns=columns)

cursor.close()
conn.close()

Preview dataset

In [4]:
df_events.tail()

,id,event_date,event_name,fighter_red,fighter_blue,round,time,weight_class,win_method,winner,stance_red,stance_blue,knockdowns_red,knockdowns_blue,sig_attempts_red,sig_attempts_blue,sig_strikes_red,sig_strikes_blue,total_strikes_attempts_red,total_strikes_attempts_blue,total_strikes_red,total_strikes_blue,sub_attempts_red,sub_attempts_blue,takedowns_red,takedowns_blue,takedown_attempts_red,takedown_attempts_blue,control_time_red,control_time_blue,head_strikes_red,head_strikes_blue,head_attempts_red,head_attempts_blue,body_strikes_red,body_strikes_blue,body_attempts_red,body_attempts_blue,leg_strikes_red,leg_strikes_blue,leg_attempts_red,leg_attempts_blue,distance_red,distance_blue,distance_attempts_red,distance_attempts_blue,clinch_strikes_red,clinch_strikes_blue,clinch_attempts_red,clinch_attempts_blue,ground_strikes_red,ground_strikes_blue,ground_attempts_red,ground_attempts_blue
8332,8333,2025-09-06,UFC Fight Night: Imavov vs. Borralho,Nassourdine Imavov,Caio Borralho,5,300,Middleweight,Decision - Unanimous,Nassourdine Imavov,Orthodox,Southpaw,0.0,0.0,162.0,166.0,81.0,66.0,170.0,190.0,89.0,89.0,0.0,0.0,0.0,0.0,0.0,5.0,29,80,53.0,39.0,118.0,109.0,14.0,14.0,26.0,38.0,14.0,13.0,18.0,19.0,79.0,59.0,160.0,159.0,2.0,7.0,2.0,7.0,0.0,0.0,0.0,0.0
8333,8334,2025-09-06,UFC Fight Night: Imavov vs. Borralho,Benoit Saint Denis,Mauricio Ruffy,2,176,Lightweight,Submission,Benoit Saint Denis,Southpaw,Orthodox,0.0,0.0,38.0,17.0,17.0,5.0,61.0,18.0,37.0,6.0,1.0,0.0,3.0,0.0,10.0,0.0,278,34,9.0,4.0,26.0,14.0,8.0,1.0,12.0,3.0,0.0,0.0,0.0,0.0,9.0,5.0,26.0,17.0,3.0,0.0,3.0,0.0,5.0,0.0,9.0,0.0
8334,8335,2025-09-06,UFC Fight Night: Imavov vs. Borralho,Axel Sola,Rhys McKee,3,122,Welterweight,KO/TKO,Axel Sola,Southpaw,Orthodox,1.0,0.0,105.0,135.0,45.0,33.0,114.0,156.0,52.0,48.0,0.0,0.0,1.0,0.0,4.0,0.0,185,0,40.0,27.0,94.0,128.0,5.0,6.0,11.0,7.0,0.0,0.0,0.0,0.0,36.0,23.0,94.0,116.0,5.0,10.0,7.0,19.0,4.0,0.0,4.0,0.0
8335,8336,2025-09-06,UFC Fight Night: Imavov vs. Borralho,William Gomis,Robert Ruchala,3,300,Featherweight,Decision - Unanimous,William Gomis,Southpaw,Switch,0.0,0.0,114.0,96.0,49.0,34.0,132.0,116.0,66.0,54.0,0.0,0.0,2.0,2.0,2.0,6.0,57,276,19.0,14.0,69.0,62.0,17.0,16.0,29.0,28.0,13.0,4.0,16.0,6.0,37.0,27.0,102.0,89.0,12.0,7.0,12.0,7.0,0.0,0.0,0.0,0.0
8336,8337,2025-09-06,UFC Fight Night: Imavov vs. Borralho,Modestas Bukauskas,Paul Craig,1,300,Light Heavyweight,KO/TKO,Modestas Bukauskas,Switch,Orthodox,0.0,0.0,36.0,14.0,22.0,6.0,44.0,39.0,29.0,30.0,0.0,0.0,0.0,0.0,0.0,2.0,110,45,18.0,0.0,30.0,4.0,0.0,2.0,0.0,3.0,4.0,4.0,6.0,7.0,10.0,5.0,17.0,13.0,0.0,1.0,0.0,1.0,12.0,0.0,19.0,0.0


In [5]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8337 entries, 0 to 8336
Data columns (total 54 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           8337 non-null   int64  
 1   event_date                   8337 non-null   object 
 2   event_name                   8337 non-null   object 
 3   fighter_red                  8316 non-null   object 
 4   fighter_blue                 8316 non-null   object 
 5   round                        8337 non-null   int64  
 6   time                         8337 non-null   int64  
 7   weight_class                 8337 non-null   object 
 8   win_method                   8337 non-null   object 
 9   winner                       8190 non-null   object 
 10  stance_red                   8295 non-null   object 
 11  stance_blue                  8261 non-null   object 
 12  knockdowns_red               8316 non-null   float64
 13  knockdowns_blue   

In [6]:
df_events.describe()

,id,round,time,knockdowns_red,knockdowns_blue,sig_attempts_red,sig_attempts_blue,sig_strikes_red,sig_strikes_blue,total_strikes_attempts_red,total_strikes_attempts_blue,total_strikes_red,total_strikes_blue,sub_attempts_red,sub_attempts_blue,takedowns_red,takedowns_blue,takedown_attempts_red,takedown_attempts_blue,control_time_red,control_time_blue,head_strikes_red,head_strikes_blue,head_attempts_red,head_attempts_blue,body_strikes_red,body_strikes_blue,body_attempts_red,body_attempts_blue,leg_strikes_red,leg_strikes_blue,leg_attempts_red,leg_attempts_blue,distance_red,distance_blue,distance_attempts_red,distance_attempts_blue,clinch_strikes_red,clinch_strikes_blue,clinch_attempts_red,clinch_attempts_blue,ground_strikes_red,ground_strikes_blue,ground_attempts_red,ground_attempts_blue
count,8337.00000,8337.000000,8337.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8337.000000,8337.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000,8316.000000
mean,4169.00000,2.353604,228.060813,0.243867,0.183381,85.061688,80.044493,39.152718,34.613516,107.661015,98.725950,58.877345,50.912458,0.437229,0.318903,1.210077,0.906806,2.957311,2.691318,150.147655,110.858462,24.859909,21.596320,65.885041,62.220899,8.069745,7.176647,11.502525,10.512025,6.223064,5.840548,7.674122,7.311568,27.991222,26.147427,69.042208,67.704786,5.186989,4.704666,7.368086,6.838985,5.974507,3.761424,8.651395,5.500722
std,2406.82893,1.017928,97.303021,0.515477,0.463112,71.532433,69.570746,33.115821,31.414506,79.761970,77.207355,45.885120,42.569967,0.876515,0.747517,1.805650,1.536596,3.757211,3.721718,190.171058,153.190488,23.079735,21.923005,58.080072,56.947735,9.015038,8.254356,12.393852,11.438233,8.080393,7.643677,9.889521,9.383599,30.121638,28.128781,69.160956,66.363023,7.670297,6.791851,10.379604,9.247108,9.828638,7.840983,14.239952,11.059512
min,1.00000,1.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2085.00000,1.000000,152.000000,0.000000,0.000000,29.000000,25.000000,14.000000,10.000000,41.000000,34.000000,22.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,3.000000,8.000000,5.000000,22.000000,18.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,6.000000,5.000000,16.000000,16.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,4169.00000,3.000000,291.000000,0.000000,0.000000,68.000000,63.000000,31.000000,26.000000,96.000000,86.000000,51.000000,42.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,71.000000,43.000000,19.000000,15.000000,52.000000,47.000000,5.000000,5.000000,8.000000,7.000000,3.000000,3.000000,4.000000,4.000000,18.000000,17.000000,47.000000,47.000000,2.000000,2.000000,4.000000,3.000000,2.000000,1.000000,3.000000,1.000000
75%,6253.00000,3.000000,300.000000,0.000000,0.000000,122.000000,117.000000,56.000000,50.000000,157.000000,146.000000,84.000000,74.000000,1.000000,0.000000,2.000000,1.000000,4.000000,4.000000,227.000000,163.000000,35.000000,31.000000,93.000000,90.000000,12.000000,10.000000,17.000000,15.000000,9.000000,8.000000,11.000000,10.000000,41.000000,38.000000,102.000000,100.000000,7.000000,6.000000,10.000000,9.000000,8.000000,4.000000,11.000000,6.000000
max,8337.00000,5.000000,1080.000000,5.000000,6.000000,744.000000,510.000000,

In [7]:
df_events.isnull().sum()

id                               0
event_date                       0
event_name                       0
fighter_red                     21
fighter_blue                    21
round                            0
time                             0
weight_class                     0
win_method                       0
winner                         147
stance_red                      42
stance_blue                     76
knockdowns_red                  21
knockdowns_blue                 21
sig_attempts_red                21
sig_attempts_blue               21
sig_strikes_red                 21
sig_strikes_blue                21
total_strikes_attempts_red      21
total_strikes_attempts_blue     21
total_strikes_red               21
total_strikes_blue              21
sub_attempts_red                21
sub_attempts_blue               21
takedowns_red                   21
takedowns_blue                  21
takedown_attempts_red           21
takedown_attempts_blue          21
control_time_red    

Convert date to datetime

In [8]:
df_events['event_date'] = pd.to_datetime(df_events['event_date'], format="%Y-%m-%d")

Find Duplicates

In [9]:
df_events[df_events.duplicated(keep=False)]

,id,event_date,event_name,fighter_red,fighter_blue,round,time,weight_class,win_method,winner,stance_red,stance_blue,knockdowns_red,knockdowns_blue,sig_attempts_red,sig_attempts_blue,sig_strikes_red,sig_strikes_blue,total_strikes_attempts_red,total_strikes_attempts_blue,total_strikes_red,total_strikes_blue,sub_attempts_red,sub_attempts_blue,takedowns_red,takedowns_blue,takedown_attempts_red,takedown_attempts_blue,control_time_red,control_time_blue,head_strikes_red,head_strikes_blue,head_attempts_red,head_attempts_blue,body_strikes_red,body_strikes_blue,body_attempts_red,body_attempts_blue,leg_strikes_red,leg_strikes_blue,leg_attempts_red,leg_attempts_blue,distance_red,distance_blue,distance_attempts_red,distance_attempts_blue,clinch_strikes_red,clinch_strikes_blue,clinch_attempts_red,clinch_attempts_blue,ground_strikes_red,ground_strikes_blue,ground_attempts_red,ground_attempts_blue


In [10]:
df_events = df_events.drop_duplicates()

Find missing values

In [11]:
missing_df = pd.DataFrame({
    'missing_count': df_events.isnull().sum(),
    'missing_percent': round((df_events.isnull().sum() / len(df_events)) * 100, 2)
}).sort_values(by='missing_percent', ascending=False)

missing_df

,missing_count,missing_percent
winner,147,1.76
stance_blue,76,0.91
stance_red,42,0.50
takedown_attempts_blue,21,0.25
distance_red,21,0.25
head_attempts_red,21,0.25
head_attempts_blue,21,0.25
body_strikes_red,21,0.25
body_strikes_blue,21,0.25
body_attempts_red,21,0.25


Remove null values from events that also on the official site do not have any stats.

Both fighters are null values.

In [12]:
df_events[(df_events['fighter_red'].isna()) & (df_events['fighter_blue'].isna())]

,id,event_date,event_name,fighter_red,fighter_blue,round,time,weight_class,win_method,winner,stance_red,stance_blue,knockdowns_red,knockdowns_blue,sig_attempts_red,sig_attempts_blue,sig_strikes_red,sig_strikes_blue,total_strikes_attempts_red,total_strikes_attempts_blue,total_strikes_red,total_strikes_blue,sub_attempts_red,sub_attempts_blue,takedowns_red,takedowns_blue,takedown_attempts_red,takedown_attempts_blue,control_time_red,control_time_blue,head_strikes_red,head_strikes_blue,head_attempts_red,head_attempts_blue,body_strikes_red,body_strikes_blue,body_attempts_red,body_attempts_blue,leg_strikes_red,leg_strikes_blue,leg_attempts_red,leg_attempts_blue,distance_red,distance_blue,distance_attempts_red,distance_attempts_blue,clinch_strikes_red,clinch_strikes_blue,clinch_attempts_red,clinch_attempts_blue,ground_strikes_red,ground_strikes_blue,ground_attempts_red,ground_attempts_blue
22,23,1994-12-16,UFC 4: Revenge of the Warriors,None,None,1,14,Open Weight,Submission,Joe Charles,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,28,1994-12-16,UFC 4: Revenge of the Warriors,None,None,1,295,Open Weight,KO/TKO,Marcus Bossett,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,45,1995-07-14,UFC 6: Clash of the Titans,None,None,1,186,Open Weight,KO/TKO,Anthony Macias,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,48,1995-07-14,UFC 6: Clash of the Titans,None,None,1,121,Open Weight,KO/TKO,Joel Sutton,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,53,1995-09-08,UFC 7: The Brawl in Buffalo,None,None,1,326,Open Weight,KO/TKO,Onassis Parungao,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,62,1995-09-08,UFC 7: The Brawl in Buffalo,None,None,1,48,Open Weight,TKO - Doctor's Stoppage,Joel Sutton,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,63,1995-12-16,UFC - Ultimate Ultimate '95,None,None,1,329,Open Weight,Submission,Mark Hall,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,71,1995-12-16,UFC - Ultimate Ultimate '95,None,None,1,278,Open Weight,Submission,Joe Charles,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,72,1996-02-16,UFC 8: David vs Goliath,None,None,1,50,Open Weight,KO/TKO,Sam Adkins,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,88,1996-07-12,UFC 10: The Tournament,None,None,2,180,Open Weight,Decision - Unanimous,Sam Adkins,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_events = df_events[~((df_events['fighter_red'].isna()) & (df_events['fighter_blue'].isna()))]

In [14]:
missing_df = pd.DataFrame({
    'missing_count': df_events.isnull().sum(),
    'missing_percent': round((df_events.isnull().sum() / len(df_events)) * 100, 2)
}).sort_values(by='missing_percent', ascending=False)

missing_df

,missing_count,missing_percent
winner,147,1.77
stance_blue,55,0.66
stance_red,21,0.25
id,0,0.00
leg_attempts_blue,0,0.00
head_strikes_blue,0,0.00
head_attempts_red,0,0.00
head_attempts_blue,0,0.00
body_strikes_red,0,0.00
body_strikes_blue,0,0.00


Drop fights where winner is null. These fight outcomes rarely happen and are usually contributed to fighters that can not continue the fight or draws.

In [15]:
df_events[df_events[['winner']].isnull().all(axis=1)].tail(10)

,id,event_date,event_name,fighter_red,fighter_blue,round,time,weight_class,win_method,winner,stance_red,stance_blue,knockdowns_red,knockdowns_blue,sig_attempts_red,sig_attempts_blue,sig_strikes_red,sig_strikes_blue,total_strikes_attempts_red,total_strikes_attempts_blue,total_strikes_red,total_strikes_blue,sub_attempts_red,sub_attempts_blue,takedowns_red,takedowns_blue,takedown_attempts_red,takedown_attempts_blue,control_time_red,control_time_blue,head_strikes_red,head_strikes_blue,head_attempts_red,head_attempts_blue,body_strikes_red,body_strikes_blue,body_attempts_red,body_attempts_blue,leg_strikes_red,leg_strikes_blue,leg_attempts_red,leg_attempts_blue,distance_red,distance_blue,distance_attempts_red,distance_attempts_blue,clinch_strikes_red,clinch_strikes_blue,clinch_attempts_red,clinch_attempts_blue,ground_strikes_red,ground_strikes_blue,ground_attempts_red,ground_attempts_blue
7391,7392,2023-11-04,UFC Fight Night: Almeida vs. Lewis,Rinat Fakhretdinov,Elizeu Zaleski dos Santos,3,300,Welterweight,Decision - Majority,None,Orthodox,Orthodox,1.0,0.0,186.0,152.0,87.0,75.0,216.0,211.0,109.0,115.0,0.0,1.0,1.0,0.0,9.0,1.0,265,84,50.0,47.0,144.0,113.0,11.0,17.0,16.0,26.0,26.0,11.0,26.0,13.0,65.0,57.0,156.0,129.0,7.0,2.0,8.0,3.0,15.0,16.0,22.0,20.0
7401,7402,2023-11-11,UFC 295: Prochazka vs. Pereira,Nazim Sadykhov,Viacheslav Borshchev,3,300,Lightweight,Decision - Majority,None,Southpaw,Orthodox,1.0,0.0,207.0,241.0,91.0,143.0,228.0,242.0,109.0,144.0,0.0,0.0,4.0,0.0,7.0,0.0,212,10,65.0,87.0,172.0,178.0,19.0,30.0,26.0,34.0,7.0,26.0,9.0,29.0,66.0,141.0,175.0,239.0,5.0,2.0,8.0,2.0,20.0,0.0,24.0,0.0
7421,7422,2023-11-18,UFC Fight Night: Allen vs. Craig,Trey Ogden,Nikolas Motta,3,191,Lightweight,Overturned,None,Orthodox,Orthodox,0.0,0.0,83.0,80.0,58.0,14.0,128.0,82.0,96.0,16.0,1.0,0.0,3.0,0.0,16.0,0.0,268,0,49.0,8.0,74.0,73.0,6.0,5.0,6.0,6.0,3.0,1.0,3.0,1.0,44.0,14.0,65.0,80.0,13.0,0.0,17.0,0.0,1.0,0.0,1.0,0.0
7487,7488,2024-02-03,UFC Fight Night: Dolidze vs. Imavov,Aliaskhab Khizriev,Makhmud Muradov,1,11,Middleweight,Could Not Continue,None,Southpaw,Orthodox,0.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7494,7495,2024-02-10,UFC Fight Night: Hermansson vs. Pyfer,Daniel Marcos,Aoriqileng,2,208,Bantamweight,Could Not Continue,None,Orthodox,Orthodox,0.0,0.0,126.0,95.0,81.0,45.0,129.0,96.0,84.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,28,38,50.0,26.0,84.0,71.0,12.0,10.0,19.0,12.0,19.0,9.0,23.0,12.0,69.0,43.0,113.0,93.0,5.0,2.0,6.0,2.0,7.0,0.0,7.0,0.0
7556,7557,2024-03-16,UFC Fight Night: Tuivasa vs. Tybura,Bryan Battle,Ange Loosa,2,60,Welterweight,Could Not Continue,None,Orthodox,Orthodox,0.0,0.0,75.0,54.0,27.0,23.0,78.0,58.0,28.0,26.0,0.0,0.0,1.0,0.0,2.0,1.0,93,0,16.0,6.0,56.0,34.0,10.0,8.0,18.0,9.0,1.0,9.0,1.0,11.0,24.0,22.0,69.0,53.0,3.0,1.0,5.0,1.0,0.0,0.0,1.0,0.0
7732,7733,2024-07-13,UFC Fight Night: Namajunas vs. Cortez,Abdul Razak Alhassan,Cody Brundage,1,37,Middleweight,Could Not Continue,None,Orthodox,Orthodox,0.0,0.0,30.0,2.0,23.0,1.0,35.0,2.0,24.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,27,3,23.0,1.0,30.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,4.0,0.0,4.0,0.0,19.0,0.0,26.0,0.0
8019,8020,2025-02-08,UFC 312: Du Plessis vs. Strickland 2,Jimmy Crute,Rodolfo Bellato,3,300,Light Heavyweight,Decision - Majority,None,Orthodox,Orthodox,1.0,0.0,209.0,141.0,106.0,89.0,247.0,148.0,139.0,95.0,0.0,0.0,0.0,1.0,3.0,2.0,232,29,93.0,36.0,190.0,76.0,8.0,24.0,10.0,29.0,5.0,29.0,9.0,36.0,77.0,85.0,170.0,136.0,11.0,4.0,13.0,5.0,18.0,0.0,26.0,0.0
8204,8205,2025-06-14,UFC Fight Night: Usman vs. Buckley,Mansur Abdul-Malik,Cody Brundage,3,36,Middleweight,Overturned,None,Orthodox,Orthodox,0.0,0.0,73.0,67.0,25.0,30.0,73.0,67.0,25.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,7,1,17.0,18.0,55.0,49.0,3.0,2.0,13.0,8.0,5.0,10.0,5.0,10.0,18.0,29.0,60.0,66.0,0.0,1.0,0.0,1.0,7.0,0.0,13.0,0.0
8208,8209,2025-06-14,UFC Fight Night: Usman vs. Buckley,Paul Craig,Rodolfo Bellato,1,2

Calculate the fights where there are no winners per year

In [16]:
# Extract year
df_events = df_events.copy()
df_events['year'] = df_events['event_date'].dt.year

# Total fights per year
total_fights = df_events.groupby('year').size()

# Total no-winner fights per year (count where winner is null)
no_winners = df_events.groupby('year')['winner'].apply(lambda x: x.isnull().sum())

# Average of no winners per year (ratio)
avg_no_winners = no_winners / total_fights

# Combine into summary DataFrame
df_summary = pd.DataFrame({
    'year': total_fights.index,
    'avg_no_winners': avg_no_winners.values,
    'total_fights': total_fights.values,
    'total_no_winners': no_winners.values
}).reset_index(drop=True)

df_summary

,year,avg_no_winners,total_fights,total_no_winners
0,1994,0.000000,29,0
1,1995,0.058824,34,2
2,1996,0.000000,36,0
3,1997,0.025641,39,1
4,1998,0.000000,21,0
5,1999,0.045455,44,2
6,2000,0.023256,43,1
7,2001,0.025000,40,1
8,2002,0.018868,53,1
9,2003,0.048780,41,2


In [17]:
df_events = df_events[~df_events[['winner']].isnull().all(axis=1)]

Fill missing stance values with the highest occuring one

In [18]:
df_events['stance_blue'].unique()

array([None, 'Southpaw', 'Orthodox', 'Open Stance', 'Sideways', 'Switch'],
      dtype=object)

In [19]:
df_events['stance_red'].unique()

array(['Southpaw', 'Orthodox', None, 'Sideways', 'Switch', 'Open Stance'],
      dtype=object)

In [20]:
df_events[df_events[['stance_blue']].isnull().all(axis=1)].tail(5)

,id,event_date,event_name,fighter_red,fighter_blue,round,time,weight_class,win_method,winner,stance_red,stance_blue,knockdowns_red,knockdowns_blue,sig_attempts_red,sig_attempts_blue,sig_strikes_red,sig_strikes_blue,total_strikes_attempts_red,total_strikes_attempts_blue,total_strikes_red,total_strikes_blue,sub_attempts_red,sub_attempts_blue,takedowns_red,takedowns_blue,takedown_attempts_red,takedown_attempts_blue,control_time_red,control_time_blue,head_strikes_red,head_strikes_blue,head_attempts_red,head_attempts_blue,body_strikes_red,body_strikes_blue,body_attempts_red,body_attempts_blue,leg_strikes_red,leg_strikes_blue,leg_attempts_red,leg_attempts_blue,distance_red,distance_blue,distance_attempts_red,distance_attempts_blue,clinch_strikes_red,clinch_strikes_blue,clinch_attempts_red,clinch_attempts_blue,ground_strikes_red,ground_strikes_blue,ground_attempts_red,ground_attempts_blue,year
3592,3593,2016-02-27,UFC Fight Night: Silva vs Bisping,David Teymur,Martin Svensson,2,86,Lightweight,KO/TKO,David Teymur,Southpaw,None,1.0,0.0,62.0,34.0,38.0,13.0,65.0,38.0,41.0,17.0,0.0,0.0,1.0,0.0,1.0,4.0,28,19,15.0,3.0,34.0,22.0,3.0,5.0,5.0,7.0,20.0,5.0,23.0,5.0,34.0,12.0,57.0,32.0,0.0,0.0,0.0,1.0,4.0,1.0,5.0,1.0,2016
3730,3731,2016-07-07,UFC Fight Night: Dos Anjos vs. Alvarez,Anthony Birchak,Dileno Lopes,3,300,Bantamweight,Decision - Split,Anthony Birchak,Orthodox,None,0.0,0.0,112.0,87.0,41.0,25.0,175.0,98.0,101.0,35.0,0.0,0.0,0.0,2.0,0.0,12.0,139,310,16.0,22.0,74.0,74.0,18.0,2.0,29.0,10.0,7.0,1.0,9.0,3.0,30.0,18.0,86.0,75.0,9.0,7.0,21.0,12.0,2.0,0.0,5.0,0.0,2016
3743,3744,2016-07-08,The Ultimate Fighter: Team Joanna vs. Team Clá...,Gray Maynard,Fernando Bruno,3,300,Featherweight,Decision - Unanimous,Gray Maynard,Orthodox,None,0.0,0.0,71.0,101.0,30.0,32.0,85.0,119.0,42.0,47.0,1.0,1.0,3.0,2.0,7.0,7.0,402,148,18.0,26.0,58.0,85.0,8.0,6.0,9.0,16.0,4.0,0.0,4.0,0.0,13.0,27.0,41.0,83.0,9.0,5.0,19.0,18.0,8.0,0.0,11.0,0.0,2016
3752,3753,2016-07-09,UFC 200: Tate vs Nunes,Sage Northcutt,Enrique Marin,3,300,Lightweight,Decision - Unanimous,Sage Northcutt,Orthodox,None,0.0,0.0,66.0,36.0,41.0,16.0,135.0,48.0,106.0,26.0,1.0,2.0,0.0,4.0,2.0,10.0,262,448,36.0,12.0,59.0,32.0,3.0,2.0,5.0,2.0,2.0,2.0,2.0,2.0,14.0,12.0,33.0,31.0,13.0,2.0,17.0,3.0,14.0,2.0,16.0,2.0,2016
3801,3802,2016-08-06,UFC Fight Night: Rodriguez vs. Caceres,Teruto Ishihara,Horacio Gutierrez,1,152,Featherweight,KO/TKO,Teruto Ishihara,Southpaw,None,1.0,0.0,25.0,18.0,13.0,5.0,25.0,18.0,13.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,6,0,7.0,1.0,17.0,14.0,3.0,0.0,5.0,0.0,3.0,4.0,3.0,4.0,8.0,5.0,19.0,18.0,0.0,0.0,0.0,0.0,5.0,0.0,6.0,0.0,2016


Build a fighter dataframe from both red and blue stance columns

In [21]:
df_fighters_red = df_events[['fighter_red', 'stance_red']].rename(
    columns={'fighter_red': 'fighter', 'stance_red': 'stance'}
)
df_fighters_blue = df_events[['fighter_blue', 'stance_blue']].rename(
    columns={'fighter_blue': 'fighter', 'stance_blue': 'stance'}
)

# Combine into one fighter dataframe
df_fighters = pd.concat([df_fighters_red, df_fighters_blue], ignore_index=True)

# Drop duplicate fighters (keep their first known stance)
df_fighters = df_fighters.drop_duplicates(subset=['fighter'], keep='first')

# Count unique stances across fighters
stance_counts = df_fighters['stance'].value_counts(dropna=True)

print("Unique stance counts across fighters:\n", stance_counts)

Unique stance counts across fighters:
 stance
Orthodox       1931
Southpaw        436
Switch          150
Open Stance       6
Sideways          3
Name: count, dtype: int64


In [22]:
top_stances = stance_counts.index[:1].tolist()

df_events['stance_red'] = df_events['stance_red'].fillna(top_stances[0])
df_events['stance_blue'] = df_events['stance_blue'].fillna(top_stances[0])

In [23]:
df_events.isnull().sum()

id                             0
event_date                     0
event_name                     0
fighter_red                    0
fighter_blue                   0
round                          0
time                           0
weight_class                   0
win_method                     0
winner                         0
stance_red                     0
stance_blue                    0
knockdowns_red                 0
knockdowns_blue                0
sig_attempts_red               0
sig_attempts_blue              0
sig_strikes_red                0
sig_strikes_blue               0
total_strikes_attempts_red     0
total_strikes_attempts_blue    0
total_strikes_red              0
total_strikes_blue             0
sub_attempts_red               0
sub_attempts_blue              0
takedowns_red                  0
takedowns_blue                 0
takedown_attempts_red          0
takedown_attempts_blue         0
control_time_red               0
control_time_blue              0
head_strik

Convert numerical values to int

In [24]:
for col in df_events.select_dtypes(include=['number']).columns:
    df_events[col] = df_events[col].astype('int64')

Convert winner column to 1 if winner is red or 0 if winner is blue

In [25]:
df_events['winner'] = (df_events['winner'] == df_events['fighter_red']).astype(int)
df_events.tail()

,id,event_date,event_name,fighter_red,fighter_blue,round,time,weight_class,win_method,winner,stance_red,stance_blue,knockdowns_red,knockdowns_blue,sig_attempts_red,sig_attempts_blue,sig_strikes_red,sig_strikes_blue,total_strikes_attempts_red,total_strikes_attempts_blue,total_strikes_red,total_strikes_blue,sub_attempts_red,sub_attempts_blue,takedowns_red,takedowns_blue,takedown_attempts_red,takedown_attempts_blue,control_time_red,control_time_blue,head_strikes_red,head_strikes_blue,head_attempts_red,head_attempts_blue,body_strikes_red,body_strikes_blue,body_attempts_red,body_attempts_blue,leg_strikes_red,leg_strikes_blue,leg_attempts_red,leg_attempts_blue,distance_red,distance_blue,distance_attempts_red,distance_attempts_blue,clinch_strikes_red,clinch_strikes_blue,clinch_attempts_red,clinch_attempts_blue,ground_strikes_red,ground_strikes_blue,ground_attempts_red,ground_attempts_blue,year
8332,8333,2025-09-06,UFC Fight Night: Imavov vs. Borralho,Nassourdine Imavov,Caio Borralho,5,300,Middleweight,Decision - Unanimous,1,Orthodox,Southpaw,0,0,162,166,81,66,170,190,89,89,0,0,0,0,0,5,29,80,53,39,118,109,14,14,26,38,14,13,18,19,79,59,160,159,2,7,2,7,0,0,0,0,2025
8333,8334,2025-09-06,UFC Fight Night: Imavov vs. Borralho,Benoit Saint Denis,Mauricio Ruffy,2,176,Lightweight,Submission,1,Southpaw,Orthodox,0,0,38,17,17,5,61,18,37,6,1,0,3,0,10,0,278,34,9,4,26,14,8,1,12,3,0,0,0,0,9,5,26,17,3,0,3,0,5,0,9,0,2025
8334,8335,2025-09-06,UFC Fight Night: Imavov vs. Borralho,Axel Sola,Rhys McKee,3,122,Welterweight,KO/TKO,1,Southpaw,Orthodox,1,0,105,135,45,33,114,156,52,48,0,0,1,0,4,0,185,0,40,27,94,128,5,6,11,7,0,0,0,0,36,23,94,116,5,10,7,19,4,0,4,0,2025
8335,8336,2025-09-06,UFC Fight Night: Imavov vs. Borralho,William Gomis,Robert Ruchala,3,300,Featherweight,Decision - Unanimous,1,Southpaw,Switch,0,0,114,96,49,34,132,116,66,54,0,0,2,2,2,6,57,276,19,14,69,62,17,16,29,28,13,4,16,6,37,27,102,89,12,7,12,7,0,0,0,0,2025
8336,8337,2025-09-06,UFC Fight Night: Imavov vs. Borralho,Modestas Bukauskas,Paul Craig,1,300,Light Heavyweight,KO/TKO,1,Switch,Orthodox,0,0,36,14,22,6,44,39,29,30,0,0,0,0,0,2,110,45,18,0,30,4,0,2,0,3,4,4,6,7,10,5,17,13,0,1,0,1,12,0,19,0,2025


Normalize weight classes

In [26]:
df_events["weight_class"].value_counts()

weight_class
Lightweight                                         1328
Welterweight                                        1275
Middleweight                                        1028
Featherweight                                        770
Bantamweight                                         685
                                                    ... 
Ultimate Fighter 6 Welterweight Tournament Title       1
Ultimate Fighter 5 Lightweight Tournament Title        1
Ultimate Fighter 4 Welterweight Tournament Title       1
Ultimate Fighter 4 Middleweight Tournament Title       1
Ultimate Fighter 33 Flyweight Tournament Title         1
Name: count, Length: 119, dtype: int64

In [27]:
def clean_text(val: str) -> str:
    if not isinstance(val, str):
        return val
    val = val.strip().lower()  # lowercase + trim
    val = re.sub(r"\s+", " ", val)  # collapse multiple spaces
    val = val.replace("womens", "women's")  # normalize missing apostrophe
    val = val.replace("women ", "women's ") # normalize if missing "'s"
    return val

# master map of cleaned values → canonical
mapping = {
    "lightweight": "Lightweight",
    "welterweight": "Welterweight",
    "middleweight": "Middleweight",
    "featherweight": "Featherweight",
    "bantamweight": "Bantamweight",
    "heavyweight": "Heavyweight",
    "light heavyweight": "Light Heavyweight",
    "flyweight": "Flyweight",
    "women's strawweight": "Women's Strawweight",
    "women's flyweight": "Women's Flyweight",
    "women's bantamweight": "Women's Bantamweight",
    "open weight": "Open Weight",
    "catch weight": "Catch Weight"
}

def normalize_weight_class(val):
    # clean first
    cleaned = clean_text(val)

    # if it matches exactly after cleaning
    if cleaned in mapping:
        return mapping[cleaned]
    
    # fuzzy keyword matching for tournament-style
    if "heavyweight" in cleaned and "light" not in cleaned:
        return "Heavyweight"
    elif "lightweight" in cleaned and "feather" not in cleaned:
        return "Lightweight"
    elif "middleweight" in cleaned:
        return "Middleweight"
    elif "featherweight" in cleaned:
        return "Featherweight"
    elif "bantamweight" in cleaned:
        # if it’s a women’s version
        if "women" in cleaned:
            return "Women's Bantamweight"
        return "Bantamweight"
    elif "flyweight" in cleaned:
        if "women" in cleaned:
            return "Women's Flyweight"
        return "Flyweight"
    elif "strawweight" in cleaned:
        return "Women's Strawweight"
    
    # default
    return "Open Weight"


In [28]:
df_events["weight_class"] = df_events["weight_class"].apply(normalize_weight_class)

In [29]:
df_events["weight_class"].value_counts()

weight_class
Lightweight             1378
Welterweight            1275
Middleweight            1082
Featherweight            832
Heavyweight              727
Bantamweight             718
Light Heavyweight        653
Flyweight                382
Women's Strawweight      345
Women's Flyweight        257
Women's Bantamweight     226
Open Weight              222
Catch Weight              72
Name: count, dtype: int64

Drop columns and reset index

In [30]:
df_events.drop(columns=['id', 'year'], axis=1, inplace=True)

df_events = df_events.reset_index(drop=True)

Normalize names for future matching

In [28]:
def normalize_name(name):

    if pd.isna(name):
        return ""
    
    # Convert to string if not already
    name = str(name)

    # Remove extra whitespace
    #name = name.strip()

    # Normalize unicode characters
    name = unicodedata.normalize('NFKD', name)
    name = ''.join(c for c in name if not unicodedata.combining(c))

    # Convert to lower
    name = name.lower()

    # Remove apostrophes and replace with nothing or space
    #name = re.sub(r"'", "", name)

    # Remove periods and other punctuation
    name = re.sub(r'[^\w\s-]', '', name)

    # Normalize spaces (multiple spaces to single space)
    name = re.sub(r'\s+', ' ', name)

    words = name.split()

    return ' '.join(words).strip()

In [32]:
df_events[["fighter_red", "fighter_blue"]] = df_events[["fighter_red", "fighter_blue"]].map(normalize_name)

In [33]:
df_events

,event_date,event_name,fighter_red,fighter_blue,round,time,weight_class,win_method,winner,stance_red,stance_blue,knockdowns_red,knockdowns_blue,sig_attempts_red,sig_attempts_blue,sig_strikes_red,sig_strikes_blue,total_strikes_attempts_red,total_strikes_attempts_blue,total_strikes_red,total_strikes_blue,sub_attempts_red,sub_attempts_blue,takedowns_red,takedowns_blue,takedown_attempts_red,takedown_attempts_blue,control_time_red,control_time_blue,head_strikes_red,head_strikes_blue,head_attempts_red,head_attempts_blue,body_strikes_red,body_strikes_blue,body_attempts_red,body_attempts_blue,leg_strikes_red,leg_strikes_blue,leg_attempts_red,leg_attempts_blue,distance_red,distance_blue,distance_attempts_red,distance_attempts_blue,clinch_strikes_red,clinch_strikes_blue,clinch_attempts_red,clinch_attempts_blue,ground_strikes_red,ground_strikes_blue,ground_attempts_red,ground_attempts_blue
0,1994-03-11,UFC 2: No Way Out,jason delucia,scott baker,1,401,Open Weight,Submission,1,Southpaw,Orthodox,0,0,5,2,3,0,25,23,20,14,5,0,0,1,1,1,0,0,1,0,2,2,1,0,2,0,1,0,1,0,2,0,4,2,0,0,0,0,1,0,1,0
1,1994-03-11,UFC 2: No Way Out,royce gracie,remco pardoel,1,91,Open Weight,Submission,1,Southpaw,Southpaw,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1994-03-11,UFC 2: No Way Out,patrick smith,johnny rhodes,1,67,Open Weight,Submission,1,Orthodox,Orthodox,0,0,12,9,5,4,12,9,5,4,1,0,0,0,0,0,0,0,1,2,4,5,2,0,2,0,2,2,6,4,3,4,10,9,2,0,2,0,0,0,0,0
3,1994-03-11,UFC 2: No Way Out,frank hamaker,thaddeus luster,1,292,Open Weight,Submission,1,Orthodox,Orthodox,0,0,3,0,2,0,15,0,14,0,3,0,1,0,1,1,0,0,2,0,3,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,2,0
4,1994-03-11,UFC 2: No Way Out,patrick smith,ray wizard,1,58,Open Weight,Submission,1,Orthodox,Orthodox,0,0,1,1,1,1,1,2,1,2,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8164,2025-09-06,UFC Fight Night: Imavov vs. Borralho,nassourdine imavov,caio borralho,5,300,Middleweight,Decision - Unanimous,1,Orthodox,Southpaw,0,0,162,166,81,66,170,190,89,89,0,0,0,0,0,5,29,80,53,39,118,109,14,14,26,38,14,13,18,19,79,59,160,159,2,7,2,7,0,0,0,0
8165,2025-09-06,UFC Fight Night: Imavov vs. Borralho,benoit saint denis,mauricio ruffy,2,176,Lightweight,Submission,1,Southpaw,Orthodox,0,0,38,17,17,5,61,18,37,6,1,0,3,0,10,0,278,34,9,4,26,14,8,1,12,3,0,0,0,0,9,5,26,17,3,0,3,0,5,0,9,0
8166,2025-09-06,UFC Fight Night: Imavov vs. Borralho,axel sola,rhys mckee,3,122,Welterweight,KO/TKO,1,Southpaw,Orthodox,1,0,105,135,45,33,114,156,52,48,0,0,1,0,4,0,185,0,40,27,94,128,5,6,11,7,0,0,0,0,36,23,94,116,5,10,7,19,4,0,4,0
8167,2025-09-06,UFC Fight Night: Imavov vs. Borralho,william gomis,robert ruchala,3,300,Featherweight,Decision - Unanimous,1,Southpaw,Switch,0,0,114,96,49,34,132,116,66,54,0,0,2,2,2,6,57,276,19,14,69,62,17,16,29,28,13,4,16,6,37,27,102,89,12,7,12,7,0,0,0,0


In [34]:
df_events.to_csv('../data/notebooks/events_cleaned.csv', index=False)

Review Stats

In [32]:
# Stats data
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    database=os.getenv("DB_NAME"),
    password=os.getenv("DB_PASSWORD")
    )

cursor = conn.cursor()

query = ("SELECT * FROM stats")

cursor.execute(query)

columns = [desc[0] for desc in cursor.description]

rows = cursor.fetchall()

df_stats = pd.DataFrame(rows, columns=columns)

cursor.close()
conn.close()

In [33]:
df_stats.tail()

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,...,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
3058,3059,Zarah Fairn,"""Infinite""",Women's Bantamweight Division,6-5-0 (W-L-D),Active,"Paris, France",Team Figueiredo,MMA,"Oct. 5, 2019",...,46.0,60.0,0.00,09:24,160.0,11.0,0.0,128.0,26.0,17.0
3059,3060,Rongzhu,None,Lightweight Division,27-6-0 (W-L-D),Active,"Sichuan, China",None,Striker,"Apr. 24, 2021",...,53.0,84.0,0.54,12:19,577.0,30.0,60.0,492.0,103.0,72.0
3060,3061,Zviad Lazishvili,None,Bantamweight Division,13-1-0 (W-L-D),Not Fighting,"Kobuleti, Georgia",Kaizen MMA,Freestyle,"Oct. 23, 2021",...,51.0,NaN,0.00,15:00,60.0,3.0,0.0,25.0,17.0,21.0
3061,3062,Zubaira Tukhugov,"""Warrior""",Lightweight Division,20-6-1 (W-L-D),Not Fighting,"USSR, Russia","Tiger Muay Thai - Phuket, Thailand",Striker,"Feb. 15, 2014",...,64.0,100.0,0.40,12:31,302.0,24.0,31.0,312.0,36.0,9.0
3062,3063,Zygimantas Ramaska,None,Featherweight Division,9-3-0 (W-L-D),Not Fighting,Lithuania,Ukmergė Judo Club,Judo,"Aug. 24, 2024",...,30.0,33.0,0.00,06:14,2.0,4.0,0.0,5.0,1.0,0.0


In [34]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3063 entries, 0 to 3062
Data columns (total 40 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               3063 non-null   int64  
 1   name                             3063 non-null   object 
 2   nickname                         1821 non-null   object 
 3   division                         2950 non-null   object 
 4   record                           3022 non-null   object 
 5   status                           3033 non-null   object 
 6   place_of_birth                   2881 non-null   object 
 7   trains_at                        988 non-null    object 
 8   fighting_style                   1029 non-null   object 
 9   octagon_debut                    3063 non-null   object 
 10  age                              2799 non-null   float64
 11  height                           2762 non-null   float64
 12  weight              

In [35]:
df_stats.describe()

,id,age,height,weight,reach,leg_reach,wins,losses,draws,wins_by_knockout,...,submission_avg,sig_strikes_defense,takedown_defense,knockdown_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
count,3063.000000,2799.000000,2762.000000,2951.000000,1928.000000,1646.000000,3063.000000,3063.000000,3063.000000,1223.000000,...,2769.000000,2757.000000,2332.000000,2769.000000,3010.000000,3010.000000,3010.000000,3010.000000,3010.000000,3010.000000
mean,1532.000000,36.100393,65.616039,157.571603,71.589471,39.903402,11.487104,4.384917,0.164218,6.297629,...,0.608696,52.114255,61.433105,0.328956,164.063455,31.208306,31.062458,142.726910,46.861794,36.745515
std,884.356263,7.115126,17.563993,51.142052,4.305493,2.615147,8.717709,3.753822,0.580659,4.062086,...,1.196997,10.485684,21.222651,1.106217,238.625588,47.989151,52.054297,198.996156,68.504406,56.801302
min,1.000000,19.000000,0.000000,0.000000,58.500000,32.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,766.500000,31.000000,67.000000,136.000000,69.000000,38.000000,6.000000,2.000000,0.000000,3.000000,...,0.000000,46.000000,48.000000,0.000000,17.000000,3.000000,0.000000,16.000000,5.000000,3.000000
50%,1532.000000,35.000000,70.000000,156.000000,72.000000,40.000000,11.000000,4.000000,0.000000,6.000000,...,0.240000,53.000000,63.000000,0.000000,73.000000,13.000000,10.000000,67.000000,21.000000,16.000000
75%,2297.500000,40.000000,73.000000,185.000000,74.500000,41.500000,16.000000,6.000000,0.000000,8.000000,...,0.820000,59.000000,75.000000,0.410000,216.000000,39.000000,39.000000,189.000000,61.000000,46.000000
max,3063.000000,80.000000,84.000000,415.000000,84.500000,74.000000,88.000000,24.000000,10.000000,28.000000,...,21.950000,100.000000,100.000000,32.140000,3222.000000,503.000000,523.000000,2363.000000,893.000000,508.000000


In [36]:
df_stats.isnull().sum()

id                                    0
name                                  0
nickname                           1242
division                            113
record                               41
status                               30
place_of_birth                      182
trains_at                          2075
fighting_style                     2034
octagon_debut                         0
age                                 264
height                              301
weight                              112
reach                              1135
leg_reach                          1417
wins                                  0
losses                                0
draws                                 0
wins_by_knockout                   1840
first_round_finishes               2183
win_by_dec                           53
win_by_sub                           53
sig_strikes_landed                  303
sig_strikes_attempted               303
takedowns_landed                   2150


Find Duplicates Names and Drop

In [37]:
df_stats[df_stats.duplicated('name', keep=False) == True]

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,...,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
394,395,Bruno Silva,"""Bulldog""",Flyweight Division,14-7-2 (W-L-D),Active,"Piracicaba, Brazil",American Top Team,Grappler,"Sep. 7, 2019",...,50.0,60.0,0.98,10:15,287.0,24.0,42.0,196.0,80.0,77.0
397,398,Bruno Silva,"""Blindado""",Middleweight Division,23-13-0 (W-L-D),Not Fighting,Brazil,Evolucao Thai - Curitiba,Striker,"Jun. 19, 2021",...,42.0,74.0,0.31,08:51,274.0,44.0,58.0,284.0,58.0,34.0
451,452,Casey Kenney,None,None,None,Not Fighting,None,None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
454,455,Casey Kenney,None,Bantamweight Division,16-4-1 (W-L-D),Active,"Portland, United States",None,MMA,"Jul. 18, 2017",...,58.0,61.0,0.00,13:48,598.0,54.0,18.0,361.0,168.0,141.0
1391,1392,Joey Gomez,"""KO King""",Bantamweight Division,6-2-0 (W-L-D),Not Fighting,"Fairfax, United States",Team Link Hooksett,None,"Jan. 18, 2016",...,55.0,50.0,0.62,12:07,47.0,12.0,0.0,36.0,9.0,14.0
1393,1394,Joey Gomez,None,Lightweight Division,7-0-0 (W-L-D),Not Fighting,"Reno, United States",None,None,"Sep. 5, 2025",...,51.0,NaN,0.00,15:00,47.0,0.0,9.0,54.0,2.0,0.0
2399,2400,Richie Vaculik,"""Vas""",Flyweight Division,10-5-0 (W-L-D),Not Fighting,"Sydney, Australia",None,None,"Dec. 7, 2013",...,57.0,32.0,0.00,11:15,132.0,23.0,11.0,107.0,31.0,28.0
2403,2404,Richie Vaculik,None,None,None,Not Fighting,Australia,None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN


Drop duplicates that have the most NaN values in the row

In [38]:
df_stats = df_stats.assign(nan_count=df_stats.isnull().sum(axis=1)) \
            .sort_values(['name', 'nan_count']) \
            .drop_duplicates('name', keep='first') \
            .drop('nan_count', axis=1)

In [39]:
df_stats[df_stats['name'] == 'Bruno Silva']

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,...,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
394,395,Bruno Silva,"""Bulldog""",Flyweight Division,14-7-2 (W-L-D),Active,"Piracicaba, Brazil",American Top Team,Grappler,"Sep. 7, 2019",...,50.0,60.0,0.98,10:15,287.0,24.0,42.0,196.0,80.0,77.0


Find missing values

In [40]:
missing_df = pd.DataFrame({
    'missing_count': df_stats.isnull().sum(),
    'missing_precent': round((df_stats.isnull().sum() / len(df_stats)) * 100, 2)
}).sort_values(by='missing_precent', ascending=False)

missing_df

,missing_count,missing_precent
first_round_finishes,2180,71.27
takedowns_landed,2147,70.19
trains_at,2072,67.73
fighting_style,2031,66.39
wins_by_knockout,1837,60.05
leg_reach,1414,46.22
nickname,1239,40.50
reach,1132,37.01
takedown_defense,728,23.80
takedowns_attempted,589,19.25


Most of the null values are from new fighters who do not have any stats yet or very old fighters

In [41]:
df_stats[df_stats[['sig_strikes_standing', 'sig_strikes_clinch', 'sig_strikes_ground', 'head_target', 'body_target', 'leg_target']].isnull().all(axis=1)]

,id,name,nickname,division,record,status,place_of_birth,trains_at,fighting_style,octagon_debut,...,sig_strikes_defense,takedown_defense,knockdown_avg,fight_time_avg,sig_strikes_standing,sig_strikes_clinch,sig_strikes_ground,head_target,body_target,leg_target
58,59,Alberta Cerra,None,None,None,Not Fighting,United States,None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
137,138,Alvaro Ivan Lopez Rodrigues,None,None,None,Not Fighting,Mexico,None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
149,150,Amir Aliakbari,None,Heavyweight Division,0-0-0 (W-L-D),Not Fighting,None,None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
246,247,Artem Frolov,None,Middleweight Division,0-0-0 (W-L-D),Not Fighting,None,None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
252,253,Asikeerbai Jinensibieke,None,None,None,Active,"Xinjiang, China",None,None,"Jun. 10, 2022",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
315,316,Bilyal Makhov,None,None,None,Retired,"USSR, Russia",None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
449,450,Carrese Archer,"""One Punch""",None,None,Not Fighting,None,None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
501,502,Chi Lewis-Parry,None,Heavyweight Division,0-0-0 (W-L-D),Not Fighting,"Hitchin, United Kingdom",None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
571,572,Cody Belisle,"""Bodacious""",None,None,Active,None,None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
593,594,Coltin Cole,"""The Truth""",None,None,Active,None,None,None,"Sep. 5, 2025",...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN


I will merge the two datasets togehter.
But first I will keep only a few features from the stats. Most of the stats are totals, such as wins by knockout, takedowns landed. 
If I use this data there will be data leakage and I will not get the results I want, because the model will already know total stats even of previous fights.

Thtat is why I will keep only the constant values and the averages such as octagon_debut, height, knowckdown_avg.

In [42]:
df_stats = df_stats[['name', 'octagon_debut', 'age', 'height', 'weight', 'reach',
    'leg_reach', 'sig_strikes_landed_per_minute', 'sig_strikes_absorbed_per_minute', 
    'takedowns_avg', 'submission_avg', 'knockdown_avg', 'fight_time_avg']]
df_stats

,name,octagon_debut,age,height,weight,reach,leg_reach,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,knockdown_avg,fight_time_avg
39,AJ Cunningham,"Mar. 2, 2024",30.0,70.0,136.0,71.0,41.0,4.0,6.0,0.00,0.52,0.00,09:34
44,AJ Dobson,"Feb. 12, 2022",31.0,73.0,185.0,76.0,43.5,4.0,5.0,1.67,0.28,0.28,10:47
45,AJ Fletcher,"Mar. 12, 2022",26.0,70.0,183.0,67.0,37.0,3.0,5.0,1.54,0.93,0.31,09:43
2,Aalon Cruz,"Jul. 30, 2019",33.0,72.0,155.0,78.0,42.0,8.0,9.0,0.00,0.00,0.85,05:54
0,Aaron Brink,"Nov. 17, 2000",48.0,75.0,231.0,NaN,NaN,3.0,6.0,0.00,0.00,0.00,01:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,Zhu Kangjie,"Nov. 23, 2024",29.0,NaN,146.0,NaN,NaN,3.0,1.0,0.33,0.00,0.67,15:00
3057,Zu Anyanwu,"Jul. 11, 2017",41.0,73.0,263.5,77.0,41.0,2.0,2.0,0.65,0.00,0.65,11:32
3061,Zubaira Tukhugov,"Feb. 15, 2014",32.0,68.0,157.5,68.0,38.5,3.0,3.0,2.13,0.00,0.40,12:31
3060,Zviad Lazishvili,"Oct. 23, 2021",31.0,66.0,135.0,69.0,37.0,4.0,6.0,0.00,0.00,0.00,15:00


In [43]:
df_stats.isnull().sum()

name                                  0
octagon_debut                         0
age                                 262
height                              298
weight                              109
reach                              1132
leg_reach                          1414
sig_strikes_landed_per_minute       292
sig_strikes_absorbed_per_minute     292
takedowns_avg                       292
submission_avg                      292
knockdown_avg                       292
fight_time_avg                       51
dtype: int64

Weight and Height have some 0 values, this does not make sense as someone can not have a value of height be equal to 0, which will impact the imputation.
I will convert to null and after impute.

In [44]:
df_stats[df_stats['height'] == 0]
df_stats[df_stats['weight'] == 0]

,name,octagon_debut,age,height,weight,reach,leg_reach,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,knockdown_avg,fight_time_avg
33,Adrian Serrano,"Jun. 9, 2000",59.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
38,Adriano Santos,"Oct. 16, 1998",NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
59,Alberto Cerro Leon,"Mar. 11, 1994",NaN,75.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
75,Alex Andrade,"Jun. 9, 2000",49.0,0.0,0.0,NaN,NaN,0.0,3.0,0.00,0.75,0.0,20:00
86,Alex Hunter,"Jul. 27, 1997",NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,Valeri Ignatov,"Mar. 5, 1999",NaN,70.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
2962,Wallid Ismail,"Feb. 7, 1997",55.0,70.0,0.0,NaN,NaN,1.0,1.0,1.85,0.46,0.0,16:15
2976,Wes Albritton,"May. 30, 1997",NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
3023,Yoshiki Takahashi,"Feb. 7, 1997",54.0,0.0,0.0,NaN,NaN,2.0,6.0,0.00,2.26,0.0,02:13


In [45]:
df_stats[['height', 'weight']] = df_stats[['height', 'weight']].replace(0, np.nan)

In [46]:
df_stats.isnull().sum()

name                                  0
octagon_debut                         0
age                                 262
height                              476
weight                              269
reach                              1132
leg_reach                          1414
sig_strikes_landed_per_minute       292
sig_strikes_absorbed_per_minute     292
takedowns_avg                       292
submission_avg                      292
knockdown_avg                       292
fight_time_avg                       51
dtype: int64

In [47]:
df_stats[df_stats['knockdown_avg'].isnull()]

,name,octagon_debut,age,height,weight,reach,leg_reach,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,knockdown_avg,fight_time_avg
3,Abdul Azeem Badakhshi,"Sep. 5, 2025",27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
21,Adam Hunter,"Sep. 5, 2025",39.0,73.0,185.5,73.0,40.0,NaN,NaN,NaN,NaN,NaN,00:00
25,Adam Khaliev,"Sep. 5, 2025",39.0,70.0,170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
27,Adli Edwards,"Sep. 5, 2025",33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
33,Adrian Serrano,"Jun. 9, 2000",59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989,William Marcario,"Sep. 5, 2025",31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2991,Willian Souza,"Sep. 5, 2025",36.0,NaN,146.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
3006,Yamato Nishikawa,"Oct. 22, 2022",20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00
785,Yuri Vaulin,"Jul. 27, 1997",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00


Fill these stats with 0 because the fighters have just signed with the UFC or are too old fighters.

In [48]:
df_stats[['sig_strikes_landed_per_minute', 'sig_strikes_absorbed_per_minute','takedowns_avg',
        'submission_avg','knockdown_avg']] = df_stats[['sig_strikes_landed_per_minute', 
                                        'sig_strikes_absorbed_per_minute','takedowns_avg','submission_avg',
                                        'knockdown_avg']].fillna(0)

In [49]:
df_stats[df_stats['fight_time_avg'].isnull()].tail(5)

,name,octagon_debut,age,height,weight,reach,leg_reach,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,knockdown_avg,fight_time_avg
2786,Testy Test,"Sep. 5, 2025",NaN,NaN,NaN,NaN,34.0,0.0,0.0,0.0,0.0,0.0,None
2822,Timo Feucht,"Sep. 5, 2025",27.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,None
2828,Timothy Thomas,"Sep. 5, 2025",34.0,NaN,214.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,None
2938,Vineesh Subrahmanyan,"Sep. 5, 2025",NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,None
2989,William Marcario,"Sep. 5, 2025",31.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,None


In [50]:
df_stats[['fight_time_avg']] = df_stats[['fight_time_avg']].fillna('00:00')

Converting fight_time_avg to seconds

In [51]:
df_stats['fight_time_avg'] = pd.to_timedelta('00:' + df_stats['fight_time_avg']).dt.total_seconds().astype(int)

In [52]:
df_stats

,name,octagon_debut,age,height,weight,reach,leg_reach,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,knockdown_avg,fight_time_avg
39,AJ Cunningham,"Mar. 2, 2024",30.0,70.0,136.0,71.0,41.0,4.0,6.0,0.00,0.52,0.00,574
44,AJ Dobson,"Feb. 12, 2022",31.0,73.0,185.0,76.0,43.5,4.0,5.0,1.67,0.28,0.28,647
45,AJ Fletcher,"Mar. 12, 2022",26.0,70.0,183.0,67.0,37.0,3.0,5.0,1.54,0.93,0.31,583
2,Aalon Cruz,"Jul. 30, 2019",33.0,72.0,155.0,78.0,42.0,8.0,9.0,0.00,0.00,0.85,354
0,Aaron Brink,"Nov. 17, 2000",48.0,75.0,231.0,NaN,NaN,3.0,6.0,0.00,0.00,0.00,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,Zhu Kangjie,"Nov. 23, 2024",29.0,NaN,146.0,NaN,NaN,3.0,1.0,0.33,0.00,0.67,900
3057,Zu Anyanwu,"Jul. 11, 2017",41.0,73.0,263.5,77.0,41.0,2.0,2.0,0.65,0.00,0.65,692
3061,Zubaira Tukhugov,"Feb. 15, 2014",32.0,68.0,157.5,68.0,38.5,3.0,3.0,2.13,0.00,0.40,751
3060,Zviad Lazishvili,"Oct. 23, 2021",31.0,66.0,135.0,69.0,37.0,4.0,6.0,0.00,0.00,0.00,900


Drop age column because the UFC website does not update it consistently and there are many wrong values especially for older fighters

In [53]:
df_stats.drop(columns='age', axis=1, inplace=True)

Impute missing values

In [54]:
print(df_stats.columns[df_stats.isnull().any()])
print("\n", len(df_stats.columns[df_stats.isnull().any()]))

Index(['height', 'weight', 'reach', 'leg_reach'], dtype='object')

 4


In [55]:
missing_cols = df_stats.columns[df_stats.isnull().any()]

imputer = KNNImputer(n_neighbors=5)
df_stats[missing_cols] = imputer.fit_transform(df_stats[missing_cols])

# Save as pickle
with open("../models/knn_imputer_stats.pkl", "wb") as f:
    pickle.dump(imputer, f)

In [56]:
df_stats.isnull().sum()

name                               0
octagon_debut                      0
height                             0
weight                             0
reach                              0
leg_reach                          0
sig_strikes_landed_per_minute      0
sig_strikes_absorbed_per_minute    0
takedowns_avg                      0
submission_avg                     0
knockdown_avg                      0
fight_time_avg                     0
dtype: int64

Some names have special characters. When I try and merge the events and stats dataframes these names will not be matched.
I need to normalize those names.

In [57]:
df_stats["name"] = df_stats["name"].map(normalize_name)

In [58]:
df_stats

,name,octagon_debut,height,weight,reach,leg_reach,sig_strikes_landed_per_minute,sig_strikes_absorbed_per_minute,takedowns_avg,submission_avg,knockdown_avg,fight_time_avg
39,aj cunningham,"Mar. 2, 2024",70.0,136.0,71.0,41.0,4.0,6.0,0.00,0.52,0.00,574
44,aj dobson,"Feb. 12, 2022",73.0,185.0,76.0,43.5,4.0,5.0,1.67,0.28,0.28,647
45,aj fletcher,"Mar. 12, 2022",70.0,183.0,67.0,37.0,3.0,5.0,1.54,0.93,0.31,583
2,aalon cruz,"Jul. 30, 2019",72.0,155.0,78.0,42.0,8.0,9.0,0.00,0.00,0.85,354
0,aaron brink,"Nov. 17, 2000",75.0,231.0,77.4,44.3,3.0,6.0,0.00,0.00,0.00,111
...,...,...,...,...,...,...,...,...,...,...,...,...
3055,zhu kangjie,"Nov. 23, 2024",67.4,146.0,69.2,39.0,3.0,1.0,0.33,0.00,0.67,900
3057,zu anyanwu,"Jul. 11, 2017",73.0,263.5,77.0,41.0,2.0,2.0,0.65,0.00,0.65,692
3061,zubaira tukhugov,"Feb. 15, 2014",68.0,157.5,68.0,38.5,3.0,3.0,2.13,0.00,0.40,751
3060,zviad lazishvili,"Oct. 23, 2021",66.0,135.0,69.0,37.0,4.0,6.0,0.00,0.00,0.00,900


In [59]:
df_stats.to_csv('../data/notebooks/stats_cleaned.csv', index = False)